In [46]:
import requests
import json
import pandas as pd

def get_api_key() -> str:
    json_file_path: str = './key.json'

    with open(json_file_path, 'r') as json_file:
        json_data: json = json.load(json_file)
        api_key_property: str = "api_key"
        json_api_key: str = json_data.get(api_key_property)
    return json_api_key

api_key: str = get_api_key()
interval: str = "1min"
order: str = "ASC"
symbol: str = "TSLA"
output: float = 5000

empty_signal: str = ""
buy_signal: str = "Buy"
wait_signal: str = "Wait"
sell_signal: str = "Sell"

open_column_name: str = "open"
close_column_name: str = "close"
bollinger_column_name: str = "percent_b"
signal_column_name: str = "Signal"
profit_absolute_column_name: str = "Profit"
profit_relative_column_name: str = "Profit (%)"

positive_trades_key: str = "Pos. trades"
negative_trades_key: str = "Neg. trades"
total_trades_key: str = "Total trades"
absolute_profits_key: str = "Total profits"
relative_profits_key: str = "Total profits (%)"



In [47]:
time_series_url: str = f"https://api.twelvedata.com/time_series?symbol={symbol}&interval={interval}&order={order}&outputsize={output}&apikey={api_key}"
dataframe_key: str = "datetime"
response_values: str = "values"

def get_api_values(url: str) -> pd.DataFrame:
    response = requests.get(url)
    values: json = response.json().get(response_values, [])
    dataframe: pd.DataFrame = pd.DataFrame(values)
    dataframe[dataframe_key] = pd.to_datetime(dataframe[dataframe_key])

    if response.status_code != 200:
        print(f"Error: {response.status_code} - {response.text}")
    else:
        return dataframe

time_series= get_api_values(time_series_url)
#time_series_data

In [48]:
bollinger_bands_url: str = f"https://api.twelvedata.com/percent_b?symbol={symbol}&interval={interval}&outputsize={output}&order={order}&apikey={api_key}"

bollinger_bands: pd.DataFrame = get_api_values(bollinger_bands_url)
# bollinger_bands


In [49]:
def get_merged_dataframes(first_dataframe: pd.DataFrame, second_dataframe: pd.DataFrame, merge_key: str) -> pd.DataFrame:
    last_datetime_first_dataframe = first_dataframe.at[0, dataframe_key]
    last_datetime_second_dataframe = first_dataframe.at[0, dataframe_key]
    merge_conflict_error_message: str = "datetime merging conflict"
    if  last_datetime_first_dataframe == last_datetime_second_dataframe:
        return pd.merge(first_dataframe, second_dataframe, left_on=merge_key, right_on=merge_key)
    else:
        print(merge_conflict_error_message)
indicators: pd.DataFrame = get_merged_dataframes(time_series, bollinger_bands, dataframe_key)
# indicators

In [50]:
def is_bollinger_entry_satisfied(current_bollinger_value: float, last_bollinger_value: float) -> bool:
    current_bollinger_value_negative: bool = current_bollinger_value < 0
    last_bollinger_value_positive: bool = last_bollinger_value >= 0
    return current_bollinger_value_negative and last_bollinger_value_positive

def is_bollinger_exit_satisfied(current_bollinger_value: float, last_bollinger_value: float) -> bool:
    current_bollinger_value_smaller_one: bool = current_bollinger_value < 1
    last_bollinger_value_greater_one: bool = last_bollinger_value >= 1
    return current_bollinger_value_smaller_one and last_bollinger_value_greater_one

def get_dataframe_row_value(dataframe_row: pd.DataFrame, column_name: str) -> float:
    return float(dataframe_row[column_name])

def get_trading_signal(dataframe_row: pd.DataFrame) -> str:
    return dataframe_row[signal_column_name]

def get_absolute_price_change(buying_price: float, latest_price: float)  -> float:
    return round(latest_price - buying_price, 4)

def get_relative_price_change(buying_price: float, latest_price: float)  -> float:
    return round((latest_price - buying_price)/buying_price * 100, 4)


def get_trading_strategy(indicator_dataframe: pd.DataFrame) -> pd.DataFrame:
    added_signal_dataframe: pd.DataFrame = indicator_dataframe
    added_column_name: str = signal_column_name

    added_signal_dataframe.at[0, added_column_name] = empty_signal
    added_signal_dataframe.at[0, profit_absolute_column_name] = ""
    added_signal_dataframe.at[0, profit_relative_column_name] = ""

    buying_price: float = 0.00


    for entry in range(1, len(added_signal_dataframe)):
        current_dataframe_row: pd.DataFrame = indicator_dataframe.loc[entry]
        last_dataframe_row: pd.DataFrame = indicator_dataframe.loc[entry -1]
        current_bollinger_value: float = get_dataframe_row_value(current_dataframe_row, bollinger_column_name)
        last_bollinger_value: float = get_dataframe_row_value(last_dataframe_row, bollinger_column_name)
        bollinger_entry_satisfied: bool = is_bollinger_entry_satisfied(current_bollinger_value, last_bollinger_value)
        bollinger_exit_satisfied: bool = is_bollinger_exit_satisfied(current_bollinger_value, last_bollinger_value)
        last_signal: str = get_trading_signal(last_dataframe_row)
        is_last_empty_signal: bool = last_signal == empty_signal
        is_last_buy_signal: bool = last_signal == buy_signal
        is_last_wait_signal: bool = last_signal == wait_signal

        if bollinger_entry_satisfied and is_last_empty_signal:
            buying_price = get_dataframe_row_value(current_dataframe_row, close_column_name)
            added_signal_dataframe.at[entry, added_column_name] = buy_signal
            added_signal_dataframe.at[entry, profit_absolute_column_name] = 0.00
            added_signal_dataframe.at[entry, profit_relative_column_name] = 0.00

        elif (is_last_buy_signal or is_last_wait_signal) and not bollinger_exit_satisfied:
            closing_price: float = get_dataframe_row_value(current_dataframe_row, close_column_name)
            added_signal_dataframe.at[entry, added_column_name] = wait_signal
            added_signal_dataframe.at[entry, profit_absolute_column_name] = get_absolute_price_change(buying_price, closing_price)
            added_signal_dataframe.at[entry, profit_relative_column_name] = get_relative_price_change(buying_price, closing_price)

        elif bollinger_exit_satisfied and (is_last_wait_signal or is_last_buy_signal):
            closing_price: float = get_dataframe_row_value(current_dataframe_row, close_column_name)
            added_signal_dataframe.at[entry, added_column_name] = sell_signal
            added_signal_dataframe.at[entry, profit_absolute_column_name] = get_absolute_price_change(buying_price, closing_price)
            added_signal_dataframe.at[entry, profit_relative_column_name] = get_relative_price_change(buying_price, closing_price)
            buying_price = 0.00

        else:
            added_signal_dataframe.at[entry, added_column_name] = empty_signal
            added_signal_dataframe.at[entry, profit_absolute_column_name] = ''
            added_signal_dataframe.at[entry, profit_relative_column_name] = ''



    return added_signal_dataframe

trading_strategy: pd.DataFrame = get_trading_strategy(indicators)
# trading_strategy

In [51]:
def get_trading_performance(signal_dataframe: pd.DataFrame) -> pd.DataFrame:
    statistics: dict = {
        positive_trades_key: 0,
        negative_trades_key: 0,
        total_trades_key: 0,
        absolute_profits_key: 0.00,
        relative_profits_key: 0.00
    }

    dataframe_columns = signal_dataframe.columns.tolist()

    buy_signals_list: list = []
    sell_signals_list: list= []

    for entry in range(len(signal_dataframe)):
        current_dataframe_row: pd.DataFrame = signal_dataframe.loc[entry]
        current_signal: str = get_trading_signal(current_dataframe_row)
        trade_started: bool = current_signal == buy_signal
        trade_ended: bool = current_signal == sell_signal
        if trade_started:
            buy_signals_list.append(current_dataframe_row)
        elif trade_ended:
            sell_signals_list.append(current_dataframe_row)

    sell_signals: pd.DataFrame = pd.DataFrame(sell_signals_list, columns=dataframe_columns, index=range(len(sell_signals_list)))

    for signal in range(len(sell_signals)):
        statistics[total_trades_key] +=1
        statistics[absolute_profits_key] += sell_signals.loc[signal, profit_absolute_column_name]
        statistics[relative_profits_key] += sell_signals.loc[signal, profit_relative_column_name]
        if sell_signals.loc[signal, profit_absolute_column_name] > 0.00:
            statistics[positive_trades_key] +=1
        else:
            statistics[negative_trades_key] +=1

    statistics: pd.DataFrame = pd.DataFrame(statistics, index=[0])
    return statistics

trading_performance: pd.DataFrame = get_trading_performance(trading_strategy)
trading_performance


,Pos. trades,Neg. trades,Total trades,Total profits,Total profits (%)
0,30,23,53,7.1499,3.2516


In [52]:
def get_trading_statistics(trading_performance: pd.DataFrame) -> pd.DataFrame:
    positive_trades: float = get_dataframe_row_value(trading_performance.iloc[0], positive_trades_key)
    print(positive_trades)
get_trading_statistics(trading_performance)

30.0
